### Get the Environment Ready

In [1]:
from IPython.core.display import clear_output
!wget https://raw.githubusercontent.com/rahulrajpr/references/main/requirements_tf_cert_exam.txt
clear_output()

In [2]:
!wget https://raw.githubusercontent.com/rahulrajpr/references/main/helper_functions.py
clear_output()

In [3]:
!pip install -r requirements_tf_cert_exam.txt
!pip install split_folders
clear_output()

> import the libraries

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import random
import tensorflow_datasets as tfds

In [5]:
print(f'tensorflow version : {tf.__version__}')
print(f'pandas version : {pd.__version__}')
print(f'numpy version : {np.__version__}')
print(f'scipy version : {scipy.__version__}')
print(f'tensorflow dataset version : {tfds.__version__}')

tensorflow version : 2.10.0
pandas version : 1.4.2
numpy version : 1.22.4
scipy version : 1.7.3
tensorflow dataset version : 4.6.0


### Download the Dataset

In [6]:
fashion = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
type(fashion)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/fashion_mnist/3.0.1.incompleteF2IJI2/fashion_mnist-train.tfrecord*...:   0%|  …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/fashion_mnist/3.0.1.incompleteF2IJI2/fashion_mnist-test.tfrecord*...:   0%|   …

Dataset fashion_mnist downloaded and prepared to ~/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


tuple

In [7]:
info = fashion[1]
info

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='~/tensorflow_datasets/fashion_mnist/3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author  

In [8]:
dataset = fashion[0]
type(dataset)

dict

In [9]:
train_ds = dataset['train']
val_ds = dataset['test']

train_ds, val_ds

(<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

### Get the Class informations

In [10]:
num_classes = info.features['label'].num_classes
num_classes

10

### Create the DataPipeline

In [11]:
BS = 32
IMG_SIZE = (28,28)
COLOR_CHANNEL = 1
SCALE = 1/255.0

INPUT_SHAPE = IMG_SIZE + (COLOR_CHANNEL,)
INPUT_SHAPE

(28, 28, 1)

In [12]:
train_ds = train_ds.shuffle(1000).batch(BS).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BS).prefetch(tf.data.AUTOTUNE)

train_ds, val_ds

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

### Create a callbacks

In [13]:
from helper_functions import create_model_checkpoint

### Model 1

- Convolutions

In [14]:
from tensorflow.keras import layers

In [15]:
inputs = tf.keras.Input(shape = INPUT_SHAPE, dtype = tf.float16, name = 'input_layer')
x = layers.Rescaling(scale = SCALE)(inputs)
x = layers.Conv2D(filters = 16, kernel_size = 5, padding = 'same', activation = 'relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(filters = 34, kernel_size = 5, padding = 'same', activation = 'relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(units = 16, activation = 'relu')(x)
outputs = layers.Dense(units = num_classes, activation = 'softmax')(x)

model_1 = tf.keras.Model(inputs = inputs, outputs = outputs, name = 'model_1')

model_1.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 rescaling (Rescaling)       (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 34)        13634     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 34)         0         
 2D)                                                       

In [16]:
history_1 = model_1.fit(train_ds, 
                        epochs = 10,
                        steps_per_epoch = len(train_ds),
                        validation_data = val_ds,
                        validation_steps = len(val_ds),
                        callbacks = [create_model_checkpoint(model_name = model_1.name)])

Epoch 1/10
1875/1875 [==============================] - 22s 7ms/step - loss: 0.4823 - accuracy: 0.8304 - val_loss: 0.3716 - val_accuracy: 0.8678
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3134 - accuracy: 0.8870 - val_loss: 0.3080 - val_accuracy: 0.8900
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2721 - accuracy: 0.9015 - val_loss: 0.2828 - val_accuracy: 0.8957
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2444 - accuracy: 0.9121 - val_loss: 0.2741 - val_accuracy: 0.9033
Epoch 5/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2254 - accuracy: 0.9170 - val_loss: 0.2694 - val_accuracy: 0.8997
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2069 - accuracy: 0.9243 - val_loss: 0.2569 - val_accuracy: 0.9095
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1916 - accuracy: 0.9293 - val_loss: 0.2517 - val_accura

In [17]:
model_1.load_weights('checkpoints/model_1.h5')

In [18]:
from helper_functions import calculate_results

In [19]:
def pred_show_results(model,val_ds):
  pred = np.argmax(model.predict(val_ds), axis = 1)
  y_true = [x.numpy() for x in val_ds.unbatch().map(lambda x,y:y)]
  results = calculate_results(y_true = y_true, y_pred = pred)

  return results

In [20]:
results_1 = pred_show_results(model_1,val_ds)
results_1

313/313 [==============================] - 1s 2ms/step


{'accuracy': 0.9101,
 'precision': 0.9097526892936517,
 'recall': 0.9101,
 'f1': 0.909466667765642}

In [ ]:
End of